<a href="https://colab.research.google.com/github/Clarice-Satiko-Aoto/b2b-analyis/blob/feat/B2B_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

In [59]:
#importar bibliotecas
import pandas as pd
import numpy as np

# bibliotecas para trabalhar offline com a biblioteca
import plotly
import plotly.offline as py
import plotly.graph_objs as go # criará de fato os gráficos
from plotly.offline import plot, iplot
import cufflinks as cf # para conectar o plotly ao pandas
cf.go_offline()
plotly.offline.init_notebook_mode(connected = True)

#warnings
import warnings
warnings.filterwarnings("ignore")

In [60]:
#Para renderizar o plotly no colab

import plotly.io as pio
# pio.renderers
pio.renderers.default = 'colab'

# Carregar arquivos

In [61]:
df_hist = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/vendas/data/tratado/historico_completo.csv",
                      parse_dates=['data'])


In [62]:
df_hist.data.max()

Timestamp('2023-12-31 00:00:00')

# Funções

In [63]:
def mudar_tipo_de_dados(dataframe, tipos_de_dados):
    """
    Muda o tipo de dados de colunas em um DataFrame.

    Parâmetros:
    - dataframe: DataFrame Pandas
    - tipos_de_dados: Lista de pares (nome da coluna, tipo de dados)
    """

    for coluna, tipo in tipos_de_dados:
        if coluna in dataframe.columns:
            dataframe[coluna] = dataframe[coluna].astype(tipo)

## Mudar tipo de dado

In [64]:
tipos_de_dados = [
    ('gerente_id', object),
    ('categoria_id', object),
    ('supervisor_id', object),
    ('funcionario_id', object),
    ('vendedor_id', object),
    ('produtto_id', object),
    ('itens_venda_id', object),
    ('cliente_id', object),
    ('vendas_id', object),
    ('geografia_id', object)
    ]

In [65]:
mudar_tipo_de_dados(df_hist, tipos_de_dados)

In [66]:
df_hist.data.min(), df_hist.data.max()

(Timestamp('2021-02-16 00:00:00'), Timestamp('2023-12-31 00:00:00'))

## Tira teima

In [94]:
df_hist.query("gerente_id==1 & categoria_id ==4 & data >'2022-12-31' & data < '2023-04-01'").sort_values('data').set_index('data').resample('M')['meta_mensal'].first().sum()

70000.0

# EDA

Quantas unidades foram vendidas durante o todo o período?

2) preço médio unitário

In [95]:
preco_medio_unit = round(df_hist['valor_unitario'].mean(),2)
preco_medio_unit

151.71

3) Qual faturamento líquido

In [96]:
faturamento_liquido = round((df_hist['item_quantidade'] * df_hist['valor_unitario'] - df_hist['valor_desconto']).sum(),2)
faturamento_liquido

8611082.84

4) ticket medio

In [97]:
nfe_unicos = df_hist['nfe'].nunique()

ticket_medio = round(faturamento_liquido/nfe_unicos,2)
ticket_medio

2509.79

5) trabalhar com metas

In [12]:
df_hist.columns

Index(['vendas_id', 'data', 'nfe', 'cliente_id', 'nf_desconto',
       'valor_liquido', 'valor_venda', 'itens_venda_id', 'produto_id',
       'item_quantidade', 'valor_unitario', 'valor_bruto', 'valor_desconto',
       'geografia_id', 'cliente', 'cidade', 'estado', 'UF', 'Regiao',
       'vendedor_id', 'supervisor_id', 'gerente_id', 'categoria_id',
       'produto_descricao', 'tamanho', 'custoUnitario', 'categoria_descricao',
       'meta_mensal', '$_Meta_por_dia_util', 'meta_diaria_ind'],
      dtype='object')

In [99]:
#Verificar os valores das metas vendedor x categoria
for ger in df_hist['gerente_id'].unique():
  gerente = df_hist.query("gerente_id == @ger")
  for cat in gerente.categoria_id.unique():
    por_categoria = gerente.query("categoria_id == @cat").set_index('data').resample('M')['meta_mensal'].first()
    print(ger)
    print(cat)
    print(por_categoria)

1
2
data
2021-02-28              NaN
2021-03-31              NaN
2021-04-30              NaN
2021-05-31              NaN
2021-06-30              NaN
2021-07-31              NaN
2021-08-31              NaN
2021-09-30              NaN
2021-10-31              NaN
2021-11-30              NaN
2021-12-31              NaN
2022-01-31     31244.000000
2022-02-28     31244.000000
2022-03-31     26896.000000
2022-04-30     31559.000000
2022-05-31     31559.000000
2022-06-30     50342.000000
2022-07-31     92704.000000
2022-08-31     92704.000000
2022-09-30     70811.000000
2022-10-31     42643.000000
2022-11-30     43243.000000
2022-12-31     34471.000000
2023-01-31     64307.692308
2023-02-28     58461.538462
2023-03-31     67230.769231
2023-04-30     58461.538462
2023-05-31     67230.769231
2023-06-30     64307.692308
2023-07-31     61384.615385
2023-08-31     67230.769231
2023-09-30     61384.615385
2023-10-31    128615.384615
2023-11-30    128615.384615
2023-12-31    122769.230769
Freq: M, Na

In [102]:
#Verificar os valores das metas e o fator vendedor
for ger in df_hist['gerente_id'].unique():
  gerente = df_hist.query("gerente_id == @ger")
  for vend in gerente.vendedor_id.unique():
    por_vendedor = gerente.query("vendedor_id == @vend").set_index('data').resample('Y').agg(
                meta_mensal_geral = ('meta_mensal', 'first' ),
                meta_mensal_ind = ('meta_diaria_ind', 'sum'),
                valor_realizado = ('valor_liquido', 'sum')
    )

    print(ger)
    print(vend)
    print(por_vendedor)

1
4.0
            meta_mensal_geral  meta_mensal_ind  valor_realizado
data                                                           
2021-12-31                NaN             0.00        4718793.0
2022-12-31       31244.000000        196518.93        5841780.0
2023-12-31       64307.692308        297741.44        3634133.0
1
11.0
            meta_mensal_geral  meta_mensal_ind  valor_realizado
data                                                           
2021-12-31                NaN             0.00        6507268.0
2022-12-31       31244.000000        203406.77        7257451.0
2023-12-31       64307.692308        478753.30        5920404.0
1
6.0
            meta_mensal_geral  meta_mensal_ind  valor_realizado
data                                                           
2021-12-31                NaN             0.00        6462856.0
2022-12-31       31244.000000        152753.46        3680295.0
2023-12-31       64307.692308        436961.00        5993071.0
1
12.0
            me

In [ ]:
# # Primeiro, filtre para manter apenas as primeiras linhas de cada mês
# df_filtrado = df_hist.groupby(['gerente_id', 'vendedor_id', df_hist['data'].dt.to_period('Y')]).first()

# # Em seguida, calcule a soma das colunas do DataFrame filtrado
# resultado = df_filtrado.groupby(['gerente_id', 'vendedor_id']).agg(
#     soma_meta_mensal_geral=('meta_mensal', 'sum'),
#     soma_meta_mensal_ind=('meta_diaria_ind', 'sum'),
#     soma_valor_realizado=('valor_liquido', 'sum')
# )


# # Redefina o índice para obter um DataFrame final
# resultado = resultado.reset_index()

# # Exiba o resultado
# print(resultado)


In [105]:
# Agrupe os dados por 'gerente_id', 'vendedor_id' e ano da data
df_agrupado = df_hist.groupby(['gerente_id', 'vendedor_id', df_hist['data'].dt.year])

# Em seguida, calcule a soma das colunas 'meta_mensal' e 'valor_liquido' para cada grupo
resultado = df_agrupado.agg(
    Soma_Meta_Mensal_Geral=('meta_mensal', 'sum'),
    Soma_Valor_Realizado=('valor_liquido', 'sum')
)

# Exiba o resultado
print(resultado)

                             Soma_Meta_Mensal_Geral  Soma_Valor_Realizado
gerente_id vendedor_id data                                              
1          1.0         2021            0.000000e+00             5185270.0
                       2022            3.483045e+07             7194154.0
                       2023            3.777029e+07             3700078.0
           4.0         2021            0.000000e+00             4718793.0
                       2022            2.566874e+07             5841780.0
                       2023            3.836213e+07             3634133.0
           6.0         2021            0.000000e+00             6462856.0
                       2022            1.987034e+07             3680295.0
                       2023            5.743794e+07             5993071.0
           7.0         2021            0.000000e+00             5661847.0
                       2022            2.758712e+07             6303345.0
                       2023           

In [39]:


# Crie uma nova coluna que contenha apenas o ano e o mês
df_hist['ano_mes'] = df_hist['data'].dt.to_period('M')

# Agora, filtre o DataFrame para manter apenas as primeiras linhas de cada mês
df_filtrado = df_hist.groupby(['gerente_id', 'vendedor_id', 'ano_mes']).first().reset_index()

# Em seguida, calcule a soma das colunas 'meta_mensal' e 'valor_liquido' para cada grupo
resultado = df_filtrado.groupby(['gerente_id', 'vendedor_id', df_filtrado['data'].dt.year]).agg(
    Soma_Meta_Mensal_Geral=('meta_mensal', 'sum'),
    soma_meta_individual=('meta_diaria_ind','sum'),
    Soma_Valor_Realizado=('valor_liquido', 'sum')
)

# Exiba o resultado
resultado

Soma_Meta_Mensal_Geral  soma_meta_individual  \
gerente_id vendedor_id data                                                 
1          1.0         2021                    0.00                  0.00   
                       2022               579420.00               4445.32   
                       2023               698615.39               5358.98   
           4.0         2021                    0.00                  0.00   
                       2022               598203.00               4587.62   
                       2023               698615.39               5358.98   
           6.0         2021                    0.00                  0.00   
                       2022               531733.00               4084.05   
                       2023               698615.39               5358.98   
           7.0         2021                    0.00                  0.00   
                       2022               550680.00               4227.59   
                       2023               698615.39               5358.98   
           11.0        2021                    0.00                  0.00   
                       2022               561541.00               4315.76   
                       2023               642000.00               4948.72   
           12.0        2021                    0.00                  0.00   
                       2022               542022.00               4155.44   
                       2023               698615.39               5358.98   
2          2.0         2021                    0.00                  0.00   
                       2022               600142.00               4622.27   
                       2023               576346.14               4423.11   
           3.0         2021                    0.00                  0.00   
                       2022               466631.00               3562.34   
                       2023               576346.14               4423.11   
           5.0         2021                    0.00                  0.00   
                       2022               602618.00               4633.06   
                       2023               539961.53               4147.46   
           8.0         2021                    0.00                  0.00   
                       2022               558196.00               4289.32   
                       2023               576346.14               4423.11   
           9.0         2021                    0.00                  0.00   
                       2022               558196.00               4289.32   
                       2023               576346.14               4423.11   
           10.0        2021                    0.00                  0.00   
                       2022               597607.00               4586.85   
                       2023               539961.53               4147.46   

                             Soma_Valor_Realizado  
gerente_id vendedor_id data                        
1          1.0         2021               51846.0  
                       2022               68226.0  
                       2023               42837.0  
           4.0         2021               42241.0  
                       2022               55268.0  
                       2023               43507.0  
           6.0         2021               54039.0  
                       2022               56186.0  
                       2023               57907.0  
           7.0         2021               49111.0  
                       2022               53044.0  
                       2023               35924.0  
           11.0        2021               42506.0  
                       2022               64625.0  
                       2023               53879.0  
           12.0        2021               57927.0  
                       2022               63009.0  
                       2023               57202.0  
2          2.0         2021               49419.0  
          

In [14]:
# Localizar os valores NaN
valores_nan = df_hist.isna()

# filtrar as linhas com NaN
linhas_com_nan = df_hist[valores_nan.any(axis=1)]
linhas_com_nan

,vendas_id,data,nfe,cliente_id,nf_desconto,valor_liquido,valor_venda,itens_venda_id,produto_id,item_quantidade,...,supervisor_id,gerente_id,categoria_id,produto_descricao,tamanho,custoUnitario,categoria_descricao,meta_mensal,$_Meta_por_dia_util,meta_diaria_ind
0,1.0,2021-02-16,SO43660,614.0,24.90,265.0,240.10,13.0,43.0,1.0,...,2.0,1,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN
2,993.0,2021-06-24,SO46324,259.0,62.80,1014.0,951.20,7650.0,43.0,1.0,...,5.0,2,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN
3,7.0,2021-02-17,SO43665,347.0,112.95,1711.0,1598.05,61.0,43.0,1.0,...,5.0,2,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN
5,12.0,2021-02-18,SO43670,266.0,26.35,602.0,575.65,112.0,43.0,1.0,...,5.0,2,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN
7,454.0,2021-04-26,SO44762,202.0,306.85,3776.0,3469.15,3627.0,43.0,1.0,...,2.0,1,2,Paeonia,M,75.0,Manga Curta,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40515,NaN,2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,4,NaN,NaN,NaN,NaN,22615.38,1076.922857,179.49
40516,NaN,2023-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,4,NaN,NaN,NaN,NaN,22615.38,1076.922857,179.49
40517,NaN,2023-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,4,NaN,NaN,NaN,NaN,22615.38,1076.922857,179.49
40518,NaN,2023-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,4,NaN,NaN,NaN,NaN,22615.38,1076.922857,179.49
